# Imports

In [54]:
import os
import requests
import json
import ibm_db
import pandas as pd
import numpy as np
# if ibm_db still fails on Windows, download https://public.dhe.ibm.com/ibmdl/export/pub/software/data/db2/drivers/odbc_cli/ and extract to the location of ibm_db.py
from io import StringIO
from dotenv import load_dotenv
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from ibm_watson_machine_learning import APIClient

# Initialize environment variables, connect to DB2, connect WML API client

In [55]:
load_dotenv()

db2_connection = ibm_db.connect(os.environ.get('DB2_CONNECTION_STRING'), '', '')

wml_credentials = {
  'url': os.environ.get('WML_URL'),
  'apikey': os.environ.get('WML_API_KEY')
}
wml_client = APIClient(wml_credentials)
wml_client.set.default_project(os.environ.get('WSTUDIO_PROJECT_ID'))

'SUCCESS'

# Define functions

cleanup_input_data: Fill dataframe's NaN with 0s.

**Parameters**

1. **data_asset_df**: dataframe: the dataframe to clean up

**Output**

**Return type**: None

In [56]:
def cleanup_input_data(data_asset_df):
  data_asset_df.fillna(0, inplace=True)

get_weather_avg_for_month: Call TWC API to get weather for particular month, the API returns a list of precipitation and temperature per hour.
For the temperature, we take an average sum/size; For precipitation, we simply get a sum.

**Parameters**

1. **row**: dataframe row: a single row from the dataframe
2. **from_month**: String: from month inclusive (with the 0 in front if applicable)
3. **to_month**: String: to month exclusive (with the 0 in front if applicable)

**Output**

**Return types**: 

1. **Series**: Series consisting of average temperature in C and total precipitation in mm

In [57]:
def get_weather_avg_for_month(row, from_month, to_month):
  geocode = row['Coordinates']
  year = row['Year']
  api_key = os.environ.get('TWC_API_KEY')
  weather_request_URL = f'https://api.weather.com/v3/wx/hod/r1/direct?geocode={geocode}&startDateTime={year}-{from_month}-01T00Z&endDateTime={year}-{to_month}-01T00Z&format=json&units=m&apiKey={api_key}'
  """
  {
    "requestedLatitude": [43.34, ...],
    ...
    "precip1Hour": [0.00, 0.00, ...],
    ...
    "temperature": [56.2, 52.3, ...],
    ...
  }
  """
  request_result = requests.get(weather_request_URL, timeout=120)
  request_result_json = request_result.json()
  precipitation_list = request_result_json['precip1Hour']
  total_precipitation = sum(precipitation_list)
  temperature_list = request_result_json['temperature']
  average_temperature = sum(temperature_list) / len(temperature_list)
  return pd.Series([average_temperature, total_precipitation])

add_weather_data: Call get_weather_avg_for_month for the months of May, June, July, August, September for every row.

**Parameters**

1. **data_asset_df**: dataframe: the dataframe where the temperature and precipitation for May, June, July, August, and September will be added

**Output**

**Return type**: None

In [58]:
def add_weather_data(data_asset_df):
  # May
  data_asset_df[['May Temperature', 'May Precipitation']] = data_asset_df.apply(get_weather_avg_for_month, axis=1, args=("05", "06"))
  # June
  data_asset_df[['June Temperature', 'June Precipitation']] = data_asset_df.apply(get_weather_avg_for_month, axis=1, args=("06", "07"))
  # July
  data_asset_df[['July Temperature', 'July Precipitation']] = data_asset_df.apply(get_weather_avg_for_month, axis=1, args=("07", "08"))
  # August
  data_asset_df[['August Temperature', 'August Precipitation']] = data_asset_df.apply(get_weather_avg_for_month, axis=1, args=("08", "09"))
  # September
  data_asset_df[['September Temperature', 'September Precipitation']] = data_asset_df.apply(get_weather_avg_for_month, axis=1, args=("09", "10"))


get_soil_type: Call soilgrids API from ISRIC.org to get the soil type for specific coordinates.
We take the one with the highest probability.

**Parameters**

1. **row**: dataframe row: a single row from the dataframe

**Output**

**Return types**: 

1. **String**: the soil type

In [59]:
def get_soil_type(row):
  lat, lng = row['Coordinates'].split(',')
  soil_type_request_URL = f'https://rest.isric.org/soilgrids/v2.0/classification/query?lon={lng}&lat={lat}&number_classes=1'
  """
  Example of JSON response:
  {
    "type": "Point",
    ...
    "wrb_class_name": "Phaeozems",
    "wrb_class_value": 20,
    "wrb_class_probability": [["Phaeozems", 40]]
  }
  """
  request_result = requests.get(soil_type_request_URL, timeout=120)
  soil_type = request_result.json()['wrb_class_name']
  return soil_type

add_soil_data: Call get_soil_type for every row.

**Parameters**

1. **data_asset_df**: dataframe: the dataframe where the soil type column will be added

**Output**

**Return type**: None

In [60]:
def add_soil_data(data_asset_df):
  data_asset_df['Soil Type'] = data_asset_df.apply(get_soil_type, axis=1)

store_updated_data_asset: Convert the dataframe back to CSV and upload as data asset to Watson Studio.

**Parameters**

1. **data_asset_df**: dataframe: the dataframe to be converted to CSV and stored as data asset
2. **data_asset_name**: String: to be used as the new name of the data asset that will be stored in Watson Studio

**Output**

**Return type**: None

In [61]:
def store_updated_data_asset(data_asset_df, data_asset_name):
  data_asset_df.to_csv(data_asset_name, encoding='utf-8', index=False)
  metadata = {
    wml_client.data_assets.ConfigurationMetaNames.NAME: data_asset_name,
    wml_client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: data_asset_name
  }
  asset_details = wml_client.data_assets.store(meta_props=metadata)

delete_data_asset: Delete the data asset from Watson Studio.

**Parameters**

1. **data_asset_uid**: String: data asset uid to be deleted

**Output**

**Return type**: None

In [62]:
def delete_data_asset(data_asset_uid):
  wml_client.data_assets.delete(data_asset_uid)

train_model: Create a simple linear regression model

**Parameters**

1. **data_asset_df**: dataframe: dataframe that will be used to fit the linear regression model

**Output**

**Return types**:

1. **sklearn/xgboost/spark model**: model with the data fitted
2. **array**: list of soil types

In [63]:
def train_model(data_asset_df):
  y = data_asset_df['Yield (bu/A)'].values.astype(np.int64)

  label_encoder = preprocessing.LabelEncoder()
  data_asset_df.drop(['Crop Type'], axis = 1, inplace=True)
  data_asset_df.drop(['Brand'], axis = 1, inplace=True)
  data_asset_df.drop(['Seed'], axis = 1, inplace=True)
  data_asset_df.drop(['Year'], axis = 1, inplace=True)
  data_asset_df.drop(['Coordinates'], axis = 1, inplace=True)
  data_asset_df.drop(['Yield (bu/A)'], axis = 1, inplace=True)
  data_asset_df['Soil Type'] = label_encoder.fit_transform(data_asset_df['Soil Type'])

  X = data_asset_df.values.astype(np.float64)

  linear_regression_model = LinearRegression()
  linear_regression_model.fit(X, y)

  return linear_regression_model, label_encoder.classes_

get_model_info_from_db: Get the stored model id and deployment id from the database

**Parameters**

1. **crop_type**: String: the crop type
2. **seed_brand**: String: the seed brand name
3. **seed_variant**: String: the seed variant name

**Output**

**Return types**:

1. **True/False**: depending if the row exists or not
2. **None or String**: if String, it will be the uid of the current WML model
3. **None or String**: if String, it will be the uid of the current WML deployment

In [64]:
def get_model_info_from_db(crop_type, seed_brand, seed_variant):
  sql_stmt = 'select MODEL_ID, DEPLOYMENT_ID from WML_MODELS where CROP_TYPE = ? and SEED_VARIANT_BRAND = ? and SEED_VARIANT_NAME = ?'
  prep_stmt = ibm_db.prepare(db2_connection, sql_stmt)
  ibm_db.bind_param(prep_stmt, 1, crop_type)
  ibm_db.bind_param(prep_stmt, 2, seed_brand)
  ibm_db.bind_param(prep_stmt, 3, seed_variant)
  ibm_db.execute(prep_stmt)
  row = ibm_db.fetch_both(prep_stmt)
  if(row):
    return True, row['MODEL_ID'], row['DEPLOYMENT_ID']
  return False, None, None

create_or_update_model: Create/Update model and create a new revision

**Parameters**

1. **crop_type**: String: the crop type
2. **seed_brand**: String: the seed brand name
3. **seed_variant**: String: the seed variant name
4. **trained_model**: sklearn/xgboost/spark model: the model that was trained with the data
5. **model_id**: None/String: WML uid of an existing model to update

**Output**

**Return types**:

1. **String**: uid of the WML model
2. **int**: the revision number of the model

In [65]:
def create_or_update_model(crop_type, seed_brand, seed_variant, trained_model, model_id):
  wml_client.set.default_space(os.environ.get('WSTUDIO_SPACE_ID'))
  if(model_id is not None):
    wml_client.repository.update_model(existing_model_id, updated_meta_props=None, update_model=trained_model)
  else:
    sofware_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
    model_metadata = {
      wml_client.repository.ModelMetaNames.NAME: f'Model for {crop_type} - {seed_brand} - {seed_variant}',
      wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
      wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
      wml_client.repository.ModelMetaNames.INPUT_DATA_SCHEMA: { 
        'id': 'model_input_schema',
        'fields': [
          {'name': 'May Temperature', 'type': 'double'},
          {'name': 'May Precipitation', 'type': 'double'},
          {'name': 'June Temperature', 'type': 'double'},
          {'name': 'June Precipitation', 'type': 'double'},
          {'name': 'July Temperature', 'type': 'double'},
          {'name': 'July Precipitation', 'type': 'double'},
          {'name': 'August Temperature', 'type': 'double'},
          {'name': 'August Precipitation', 'type': 'double'},
          {'name': 'September Temperature', 'type': 'double'},
          {'name': 'September Precipitation', 'type': 'double'},
          {'name': 'Soil Type', 'type': 'int'}
        ]
      },
      wml_client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: { 
        'id': 'model_output_schema',
        'fields': [
          {
            'name': 'Yield (bu/A)',
            'type': 'int',
            'metadata': {
              'modeling_role': 'prediction'
            }
          }
        ]
      }
    }
    stored_model = wml_client.repository.store_model(model=trained_model, meta_props=model_metadata)
    model_id = wml_client.repository.get_model_id(stored_model)

  created_model_revision = wml_client.model_definitions.create_revision(model_id)
  """
  Example JSON for model_definition_revision:
  {
    'metadata': {
      ...
      'asset_type': 'wml_model',
      'created_at': '2022-10-26T06:57:50Z',
      'last_updated_at': '2022-10-26T07:02:05Z',
      'revision_id': 3,
      'name': 'Model for Testing',
      ...
    }
  }
  """
  model_revision_id = created_model_revision['metadata']['revision_id']

  return model_id, model_revision_id


create_or_update_deployment: Create/Update the online deployment

**Parameters**

1. **crop_type**: String: the crop type
2. **seed_brand**: String: the seed brand name
3. **seed_variant**: String: the seed variant name
4. **model_id**: String: WML uid of the model to deploy
5. **model_revision_id**: int: WML uid of the revision of the model
6. **deployment_id**: None/String: WML uid of an existing deployment to update

**Output**

**Return types**:

1. **String**: uid of the WML deployment

In [66]:
def create_or_update_deployment(crop_type, seed_brand, seed_variant, model_id, model_revision_id, deployment_id):
  wml_client.set.default_space(os.environ.get('WSTUDIO_SPACE_ID'))
  if(deployment_id is not None):
    deployment_metadata = {
      client.deployments.ConfigurationMetaNames.ASSET: {
        'id': model_id,
        'rev': model_revision_id
      }
    }
    updated_deployment = client.deployments.update(deployment_id, changes=deployment_metadata)
  else:
    deployment_metadata = {
      wml_client.deployments.ConfigurationMetaNames.NAME: f'Deployment for {crop_type} - {seed_brand} - {seed_variant}',
      wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    created_deployment = wml_client.deployments.create(model_id, meta_props=deployment_metadata, rev_id=str(model_revision_id))
    deployment_id = wml_client.deployments.get_id(created_deployment)

  return deployment_id

update_model_info_in_db: Upsert the model id and deployment id in the database

**Parameters**

1. **info_exists_in_db**: True/False: depending it the entry exists in the database or not
2. **crop_type**: String: the crop type
3. **seed_brand**: String: the seed brand name
4. **seed_variant**: String: the seed variant name
5. **model_id**: String: WML uid of the model
6. **deployment_id**: String: WML uid of the deployment
7. **soil_type_list**: String: list of soil types separated by comma

**Output**

**Return type**: None

In [67]:
def update_model_info_in_db(info_exists_in_db, crop_type, seed_brand, seed_variant, model_id, deployment_id, soil_type_list):
  soil_types = ','.join(str(x) for x in soil_type_list)
  
  if(info_exists_in_db):
    sql_stmt = 'update WML_MODELS set MODEL_ID = ?, DEPLOYMENT_ID = ?, MODEL_SOIL_TYPES = ?, MODDATE = current timestamp where CROP_TYPE = ? and SEED_VARIANT_BRAND = ? and SEED_VARIANT_NAME = ?'
    prep_stmt = ibm_db.prepare(db2_connection, sql_stmt)
    ibm_db.bind_param(prep_stmt, 1, model_id)
    ibm_db.bind_param(prep_stmt, 2, deployment_id)
    ibm_db.bind_param(prep_stmt, 3, soil_types)
    ibm_db.bind_param(prep_stmt, 4, crop_type)
    ibm_db.bind_param(prep_stmt, 5, seed_brand)
    ibm_db.bind_param(prep_stmt, 6, seed_variant)
    ibm_db.execute(prep_stmt)
  else:
    sql_stmt = 'insert into WML_MODELS (MODEL_ID, DEPLOYMENT_ID, MODEL_SOIL_TYPES, CROP_TYPE, SEED_VARIANT_BRAND, SEED_VARIANT_NAME, MODDATE) values (?, ?, ?, ?, ?, ?, current timestamp)'
    prep_stmt = ibm_db.prepare(db2_connection, sql_stmt)
    ibm_db.bind_param(prep_stmt, 1, model_id)
    ibm_db.bind_param(prep_stmt, 2, deployment_id)
    ibm_db.bind_param(prep_stmt, 3, soil_types)
    ibm_db.bind_param(prep_stmt, 4, crop_type)
    ibm_db.bind_param(prep_stmt, 5, seed_brand)
    ibm_db.bind_param(prep_stmt, 6, seed_variant)
    ibm_db.execute(prep_stmt)

process_new_csv: Get the content of the data asset and convert to a dataframe. Do some cleanup, add the weather data, add the soil type, then upload the complete CSV to Watson Studio, and delete the original CSV so we do not process it again.

**Parameters**

1. **data_asset_uid**: String: the uid of the data asset to get the content from
2. **data_asset_name**: String: the name of the data asset

**Output**

**Return type**: None

In [68]:
def process_new_csv(data_asset_uid, data_asset_name):
  data_asset_content = wml_client.data_assets.get_content(data_asset_uid)
  data_asset_content_str = data_asset_content.decode('utf-8')
  data_asset_df = pd.read_table(StringIO(data_asset_content_str), sep=',')

  crop_type = data_asset_df['Crop Type'].iloc[0]
  seed_brand = data_asset_df['Brand'].iloc[0]
  seed_variant = data_asset_df['Seed'].iloc[0]

  cleanup_input_data(data_asset_df)
  add_weather_data(data_asset_df)
  add_soil_data(data_asset_df)
  store_updated_data_asset(data_asset_df, data_asset_name.removeprefix('New'))
  delete_data_asset(data_asset_uid)

  model, soil_type_list = train_model(data_asset_df)

  info_exists_in_db, existing_model_id, existing_deployment_id = get_model_info_from_db(crop_type, seed_brand, seed_variant)
  model_id, model_revision_id = create_or_update_model(crop_type, seed_brand, seed_variant, model, existing_model_id)
  deployment_id = create_or_update_deployment(crop_type, seed_brand, seed_variant, model_id, model_revision_id, existing_deployment_id)
  update_model_info_in_db(info_exists_in_db, crop_type, seed_brand, seed_variant, model_id, deployment_id, soil_type_list)

# Loop through data assets and process the CSVs

### First we get list of data assets

In [69]:
data_assets_details = wml_client.data_assets.get_details()
wml_client.data_assets.list()

-------------------------------  ----------  ----  ------------------------------------
NAME                             ASSET_TYPE  SIZE  ASSET_ID
SeedData_Corn_C478DP.csv         data_asset  3335  1170c1a8-8bfb-4559-9159-f589fd2e05ea
NewSeedData_Corn_T6791 VT2P.csv  data_asset  899   66d85d02-a02a-4b9d-977f-536f8be243d2
-------------------------------  ----------  ----  ------------------------------------


### Then we loop to see which one starts with NewSeedData_ and process them

In [70]:
"""
Example of JSON for data_assets_details
{
  "resources":[
    {
      "metadata":{
        "project_id":"b2843657-348d-43e9-89ce-204399a0e0bc",
        ...
        "name":"SeedVariantData_Early Maturity Corn - DS-4014Q.csv",
        "asset_type":"data_asset",
        ...
        "resource_key":"SeedVariantData_Early Maturity Corn - DS-4014Q.csv",
        ...
        "asset_id":"af25dcd9-86f5-401e-b178-51e332e9f684",
        ...
        "guid":"af25dcd9-86f5-401e-b178-51e332e9f684",
        "href":"/v2/assets/af25dcd9-86f5-401e-b178-51e332e9f684?project_id=b2843657-348d-43e9-89ce-204399a0e0bc",
        "last_updated_at":"2022-10-23T07:55:35Z"
      }
    }
  ]
}
"""
data_asset_metadata_list = data_assets_details['resources']
for data_asset_metadata_obj in data_asset_metadata_list:
  data_asset_metadata = data_asset_metadata_obj['metadata']
  data_asset_name = data_asset_metadata['name']
  # only processing data assets CSVs where the name starts with NewSeedData_
  if(data_asset_name.startswith('NewSeedData_') and data_asset_name.endswith('.csv')):
    process_new_csv(data_asset_metadata['asset_id'], data_asset_name)

Creating data asset...
SUCCESS
Unsetting the project_id ...
Creating model_definition revision...
DONE


#######################################################################################

Synchronous deployment creation for uid: '67a02bc2-bb53-46a1-9eeb-f03c208ffa4d' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='bf26766e-120c-46e7-abfb-a8b7aa48b17a'
------------------------------------------------------------------------------------------------




# Cleanup

In [71]:
ibm_db.close(db2_connection)

True